In [3]:
import pandas as pd


In [2]:
df=pd.read_csv('your_output_file.csv')
df.head()


,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race,Education,latitude,longitude
0,6/7/2017 11:33:27,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000,...,1,0,0,0,0,0,NaN,NaN,37.486324,-122.232523
1,6/10/2017 17:11:29,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0,...,2,0,0,0,0,0,NaN,NaN,37.779259,-122.419329
2,6/11/2017 14:53:57,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000,...,3,0,0,0,0,0,NaN,NaN,47.603832,-122.330062
3,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000,...,7,0,0,0,0,0,NaN,NaN,37.368830,-122.036349
4,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0,...,9,0,0,0,0,0,NaN,NaN,37.389389,-122.083210


In [8]:
import pandas as pd
import dash
from dash import dcc, html, dash_table
import altair as alt
from vega_datasets import data

# Ensure latitude and longitude are numeric
df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')

# Group by location to compute average salary
grouped = df.groupby(['latitude', 'longitude']).agg(
    avg_salary=('totalyearlycompensation', 'mean'),
    location=('location', 'first')
).reset_index()

# Load world map data
world_map = alt.topo_feature(data.world_110m.url, "countries")

# Create Altair map visualization
def create_altair_map():
    base = alt.Chart(world_map).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).properties(
        width=1200,
        height=750
    ).project("equirectangular")

    points = alt.Chart(grouped).mark_circle().encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("avg_salary:Q", scale=alt.Scale(range=[10, 250]), title="Average Salary"),  # Size based on salary
        color=alt.Color("avg_salary:Q", scale=alt.Scale(scheme="redyellowblue"), title="Average Salary"),  # Color based on salary
        tooltip=["location:N", "avg_salary:Q"]
    )

    return (base + points).interactive()

# Save the Altair chart as an HTML file
altair_chart = create_altair_map()
altair_chart.save("altair_map.html")

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Salary Distribution Dashboard", style={'textAlign': 'center'}),
    
    # Embed the Altair chart using an HTML file
    html.Iframe(
        id="altair-map",
        srcDoc=open("altair_map.html", "r").read(),
        style={"width": "100%", "height": "600px", "border": "none"}
    ),
    
    # Data Table
    html.H3("Location-wise Salary Data", style={'textAlign': 'center'}),
    dash_table.DataTable(
        columns=[
            {"name": "Latitude", "id": "latitude"},
            {"name": "Longitude", "id": "longitude"},
            {"name": "Average Salary", "id": "avg_salary", "type": "numeric", "format": {"specifier": "$,.2f"}},
        ],
        data=grouped.to_dict("records"),
        style_table={'overflowX': 'auto'},
        page_size=10
    )
])

# Run the dashboard
if __name__ == "__main__":
    app.run_server(debug=True)






c:\Users\BO\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning:

the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.

